In [9]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from urllib import urlretrieve
import cPickle as pickle
import os
import gzip

import numpy as np
import theano

import lasagne
from lasagne import layers
from lasagne.updates import adam

from nolearn.lasagne import NeuralNet
from nolearn.lasagne import visualize

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
def contracting_block(layer):
    #ugotovi num_filters, padding
    layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(3, 3, 3), stride=2)
    tmp_layer = layer
    #ugotovi primeren batch norm layer
    layer = layers.BatchNormLayer(layer)
    #ugotovi primeren prelu layer
    layer = layers.prelu(layer)
    #ugotovi num_filters, padding
    layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(3, 3, 3), stride=1)
    #ugotovi pravilen merge function
    layer = layers.ElemwiseMergeLayer([layer, tmp_layer], add)
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    return layer
    

In [4]:
def start(input):
    layer = layers.Conv3DLayer(input, num_filters=1, filter_size=(3, 3, 3), stride=1)
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    return layer

In [8]:
def expanding_block(layer):
    # dodaj feature merging block
    layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(1, 1, 1), stride=1)
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    # kako deluje deconvolutional layer
    layer = layers.Conv3DLayer(input, num_filters=1, filter_size=(3, 3, 3), stride=0.5)
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(1, 1, 1), stride=1)
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    return layer

In [ ]:
#input_var = T.tensor4('X')
#target_var = T.tensor4('y')

#input = lasagne.layers.InputLayer((None, 3, 32, 32), input_var)
layer = start(input)
layer = contracting_block(layer)
layer = expanding_block(layer)

prediction = lasagne.layers.get_output(layer)
#uporabiti pravi loss metric
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
#loss = loss.mean() + 1e-4 * lasagne.regularization.regularize_network_params(
#        network, lasagne.regularization.l2)

# create parameter update expressions
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.adam(loss, params, learning_rate=0.01,
                                            momentum=0.9)

# compile training function that updates parameters and returns training loss
train_fn = theano.function([input_var, target_var], loss, updates=updates)

# train network (assuming you've got some training data in numpy arrays)
for epoch in range(100):
    loss = 0
    for input_batch, target_batch in training_data:
        loss += train_fn(input_batch, target_batch)
    print("Epoch %d: Loss %g" % (epoch + 1, loss / len(training_data)))